# Traducción de Texto Generado por RNN a Inglés con KerasNLP

**Autoras:** Alina Rojas y Mar Iborra<br>
**Basado en el trabajo original de:** [Abheesht Sharma](https://github.com/abheesht17/)<br>
**Fecha de creación de la adaptación:** 2023/01/11<br>
**Última modificación:** 2024/01/12<br>
**Descripción:**  Este proyecto integra un modelo de red neuronal recurrente (RNN) ya entrenado para la generación de texto en español, basado en el estilo de Ibai Llanos, y un modelo Transformer para su traducción al inglés, utilizando KerasNLP, ya analizado anteriormente. El objetivo es demostrar la capacidad de generar y traducir texto automáticamente, combinando ambos modelos en un flujo de trabajo de procesamiento de lenguaje natural.

## Introducción

En la primera parte de este proyecto, hemos desarrollado y entrenado un modelo de red neuronal recurrente (RNN) diseñado para imitar el estilo de Ibai Llanos en la generación de texto en español. Este modelo, ya entrenado, se encuentra disponible para su uso en el presente notebook, permitiendo generar automáticamente texto en español.

En las siguientes fases del proyecto, nos centramos en la tarea de traducción automática de textos del español al inglés. Para ello, comparamos dos enfoques: un modelo Transformer secuencia a secuencia tradicional y otro que emplea las avanzadas capas proporcionadas por KerasNLP. La segunda y tercera parte del proyecto se dedican a analizar y contrastar estos dos modelos, evaluando sus capacidades y rendimientos.

Tras un análisis exhaustivo, optamos por utilizar el modelo de traducción basado en KerasNLP. Esta decisión se fundamenta en los mejores resultados obtenidos con este enfoque, especialmente en términos de precisión y coherencia en la traducción de textos.

En este notebook, combinamos el trabajo realizado en las fases anteriores del proyecto. Generamos pequeños textos en español utilizando nuestro modelo RNN ya entrenado y, a continuación, los traducimos al inglés utilizando el eficiente modelo de traducción secuencia a secuencia basado en KerasNLP. Este enfoque integrado demuestra cómo se pueden unir distintos modelos de aprendizaje profundo para crear una solución completa de procesamiento de lenguaje natural, capaz de generar y traducir textos de manera automática.

## Configuración

In [1]:
!pip install -q --upgrade rouge-score
!pip install -q --upgrade keras-nlp
!pip install --upgrade tensorflow

In [2]:
import keras_nlp
import pathlib
import random

import keras

import tensorflow as tf
import tensorflow.data as tf_data
from tensorflow.keras import backend as K
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

Using TensorFlow backend


Definamos los parámetros e hiperparámetros para el proyecto.

In [3]:
BATCH_SIZE = 64
EPOCHS = 30
MAX_SEQUENCE_LENGTH = 50
ENG_VOCAB_SIZE = 15000
SPA_VOCAB_SIZE = 15000

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

## Descargando los Datos

In [4]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

## Procesando los Datos

In [5]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    eng = eng.lower()
    spa = spa.lower()
    text_pairs.append((spa, eng))

Así es como se ven nuestras parejas de oraciones:

In [6]:
for _ in range(5):
    print(random.choice(text_pairs))

('en cuanto dejó de llover, apareció un bonito arco iris.', 'as soon as it stopped raining a beautiful rainbow appeared.')
('hicieron blanco tres veces.', 'they hit the mark three times.')
('¿puedes comer con palillos chinos?', 'do you know how to eat with chopsticks?')
('el profesor me advirtió que no lo hiciera otra vez.', 'my teacher warned me not to do it again.')
('por favor ten paciencia.', 'please be patient.')


In [7]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


## Tokenización de los Datos


In [8]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf_data.Dataset.from_tensor_slices(text_samples)
    vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(2),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab

In [9]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

spa_samples = [text_pair[0] for text_pair in train_pairs]
spa_vocab = train_word_piece(spa_samples, SPA_VOCAB_SIZE, reserved_tokens)

eng_samples = [text_pair[1] for text_pair in train_pairs]
eng_vocab = train_word_piece(eng_samples, ENG_VOCAB_SIZE, reserved_tokens)

Vamos a ver algunos tokens.

In [10]:
print("Spanish Tokens: ", spa_vocab[100:110])
print("English Tokens: ", eng_vocab[100:110])

Spanish Tokens:  ['con', 'qué', 'le', 'ella', 'te', 'para', 'mary', 'las', 'más', 'al']
English Tokens:  ['know', 'him', 'there', 'they', 'go', 'her', 'has', 're', 'will', 'll']


In [11]:
spa_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=spa_vocab, lowercase=False
)

eng_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=eng_vocab, lowercase=False
)

In [12]:
spa_input_ex = text_pairs[0][0]
spa_tokens_ex = spa_tokenizer.tokenize(spa_input_ex)
print("Spanish sentence: ", spa_input_ex)
print("Tokens: ", spa_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    spa_tokenizer.detokenize(spa_tokens_ex),
)

print()

eng_output_ex = text_pairs[0][1]
eng_tokens_ex = eng_tokenizer.tokenize(eng_output_ex)
print("English sentence: ", eng_output_ex)
print("Tokens: ", eng_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    eng_tokenizer.detokenize(eng_tokens_ex),
)

Spanish sentence:  ni puedo ir, ni quiero.
Tokens:  tf.Tensor([298 129 141  13 298 122  15], shape=(7,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'ni puedo ir , ni quiero .', shape=(), dtype=string)

English sentence:  i can't go, nor do i want to.
Tokens:  tf.Tensor([  35   87    8   46  104   10 1762   76   35   95   66   12], shape=(12,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b"i can ' t go , nor do i want to .", shape=(), dtype=string)


In [13]:
def preprocess_batch(spa, eng):
    # Tokenizar las oraciones en español e inglés
    spa = spa_tokenizer(spa)
    eng = eng_tokenizer(eng)

    # Agregar tokens especiales ("[START]" y "[END]") a `eng` y padear.
    eng_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=eng_tokenizer.token_to_id("[START]"),
        end_value=eng_tokenizer.token_to_id("[END]"),
        pad_value=eng_tokenizer.token_to_id("[PAD]"),
    )
    eng = eng_start_end_packer(eng)

    # Padear `spa` hasta `MAX_SEQUENCE_LENGTH`.
    spa_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=spa_tokenizer.token_to_id("[PAD]"),
    )
    spa = spa_start_end_packer(spa)

    return (
        {
            "encoder_inputs": spa,
            "decoder_inputs": eng[:, :-1],
        },
        eng[:, 1:],
    )

def make_dataset(pairs):
    spa_texts, eng_texts = zip(*pairs)
    spa_texts = list(spa_texts)
    eng_texts = list(eng_texts)
    dataset = tf_data.Dataset.from_tensor_slices((spa_texts, eng_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()

# Crear datasets de entrenamiento y validación
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Echemos un vistazo rápido a las formas de las secuencias (tenemos lotes de 64 pares y todas las secuencias tienen 40 pasos de longitud):

In [14]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 50)
inputs["decoder_inputs"].shape: (64, 50)
targets.shape: (64, 50)


In [15]:
# Encoder
encoder_inputs = keras.Input(shape=(None,), name="encoder_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=SPA_VOCAB_SIZE,  # Usar el tamaño del vocabulario español
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(encoder_inputs)

encoder_outputs = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


# Decoder
decoder_inputs = keras.Input(shape=(None,), name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=ENG_VOCAB_SIZE,  # Usar el tamaño del vocabulario inglés
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(decoder_inputs)

x = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(ENG_VOCAB_SIZE, activation="softmax")(x)  # Salida en inglés
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

## Entrenando nuestro modelo

Utilizaremos la precisión como una forma rápida de monitorear el progreso del entrenamiento en los datos de validación. Es importante destacar que la traducción automática generalmente utiliza puntajes BLEU, así como otras métricas, en lugar de la precisión. Sin embargo, para utilizar métricas como ROUGE, BLEU, etc., deberíamos decodificar las probabilidades y generar el texto. La generación de texto es computacionalmente costosa y realizarla durante el entrenamiento no es recomendable.

In [16]:
transformer.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 token_and_position_embeddi  (None, None, 256)            3852800   ['encoder_inputs[0][0]']      
 ng (TokenAndPositionEmbedd                                                                       
 ing)                                                                                             
                                                                                                  
 decoder_inputs (InputLayer  [(None, None)]               0         []                  

In [17]:
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [18]:
transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 129s 92ms/step - loss: 0.8278 - accuracy: 0.8753 - val_loss: 0.5931 - val_accuracy: 0.9005
Epoch 2/30
1302/1302 [==============================] - 114s 88ms/step - loss: 0.5138 - accuracy: 0.9134 - val_loss: 0.4160 - val_accuracy: 0.9275
Epoch 3/30
1302/1302 [==============================] - 115s 88ms/step - loss: 0.4012 - accuracy: 0.9292 - val_loss: 0.3559 - val_accuracy: 0.9359
Epoch 4/30
1302/1302 [==============================] - 115s 88ms/step - loss: 0.3459 - accuracy: 0.9367 - val_loss: 0.3247 - val_accuracy: 0.9406
Epoch 5/30
1302/1302 [==============================] - 117s 90ms/step - loss: 0.3111 - accuracy: 0.9416 - val_loss: 0.3093 - val_accuracy: 0.9431
Epoch 6/30
1302/1302 [==============================] - 119s 91ms/step - loss: 0.2860 - accuracy: 0.9454 - val_loss: 0.3063 - val_accuracy: 0.9443
Epoch 7/30
1302/1302 [==============================] - 116s 89ms/step - loss: 0.2663 - accuracy: 0.9484 - val_loss: 0

## Generación y Traducción de Texto

En esta sección del proyecto, procedemos a traducir los textos generados por nuestro modelo RNN que imita el estilo discursivo de Ibai Llanos.

Comenzamos cargando el modelo RNN previamente entrenado:

In [19]:
ibai_model = tf.saved_model.load('./one_step_ibai')

#### Función para Generar Texto con el Modelo RNN:

La función `generate_text` es un componente clave en nuestro proyecto, diseñada para generar texto utilizando el modelo RNN previamente entrenado. Esta función se enfoca en la creación de texto en el estilo de Ibai Llanos, a partir de una cadena de inicio dada. A continuación, se desglosan sus características y funcionamiento:

- **Inicialización**: La función comienza inicializando los estados del modelo RNN y estableciendo el texto de inicio. Esto se realiza mediante `tf.constant([start_string])`, donde `start_string` es una cadena proporcionada por el usuario que guiará la dirección del texto generado.

- **Generación de Caracteres**: A través de un bucle `for`, la función genera caracteres uno a uno, hasta alcanzar el número deseado (`num_generate`). En cada iteración, se llama al método `generate_one_step` del modelo RNN, que predice el siguiente carácter basándose en el carácter actual y el estado del modelo.

- **Construcción del Texto**: Los caracteres generados se van acumulando en la lista `result`. Una vez completada la generación, estos caracteres se unen utilizando `tf.strings.join(result)`, formando una cadena de texto completa.

- **Decodificación y Resultado Final**: El texto generado, aún en formato de tensor, se decodifica a una cadena UTF-8 utilizando `.numpy().decode("utf-8")`, lo que permite visualizar el resultado final en un formato legible y coherente.

In [20]:
def generate_text(model, start_string, num_generate=30):
    # Inicializa los estados del modelo y el texto de inicio.
    states = None
    next_char = tf.constant([start_string])
    result = [next_char]

    # Genera caracteres hasta alcanzar el límite num_generate.
    for n in range(num_generate):
        next_char, states = model.generate_one_step(next_char, states=states)
        result.append(next_char)

    # Une los caracteres generados y decodifica el resultado.
    return tf.strings.join(result)[0].numpy().decode("utf-8")

#### Función para Traducir el Texto Generado:

La función `decode_sequences` traduce el texto generado por el modelo RNN del español al inglés mediante el uso del modelo Transformer implementado con KerasNLP. Esta función encapsula el proceso de traducción automática, y sus principales características son:

- **Tokenización de Entrada**: La función comienza tokenizando las oraciones de entrada en español. Utiliza `spa_tokenizer(input_sentences).to_tensor()` para convertir las oraciones en tokens que el modelo Transformer puede procesar.

- **Aplicación de Padding**: Para garantizar que todas las secuencias tengan la misma longitud, se aplica padding a las secuencias más cortas que `MAX_SEQUENCE_LENGTH`. Esto se logra mediante el uso de `tf.fill` y `tf.concat`.

- **Función de Decodificación**: Se define una función interna `next` que calcula los logits para el siguiente token en la secuencia, basándose en la entrada del codificador y el prompt actual. Este paso es crucial para determinar el siguiente token más probable en la secuencia de salida.

- **Construcción de Prompt**: Se construye un prompt inicial que incluye un token de inicio y tokens de padding para guiar la generación de la secuencia de salida en inglés. Esto se hace mediante `tf.fill` y concatenando el token de inicio y los tokens de padding.

- **Generación de Tokens con Sampler**: Se utiliza un `GreedySampler` de KerasNLP para generar tokens uno a uno en la secuencia de salida. Este sampler elige el token más probable en cada paso, construyendo así la traducción final.

- **Detokenización y Resultado Final**: Finalmente, los tokens generados se convierten de nuevo en texto legible mediante el `detokenize` del tokenizador de inglés, proporcionando así la traducción final en inglés.

In [21]:
def decode_sequences(input_sentences):
    batch_size = len(input_sentences)

    # Tokenizar la entrada del codificador (en español).
    encoder_input_tokens = spa_tokenizer(input_sentences).to_tensor()

    # Aplicar padding si la longitud de la secuencia es menor que MAX_SEQUENCE_LENGTH.
    sequence_length = tf.shape(encoder_input_tokens)[1]
    if sequence_length < MAX_SEQUENCE_LENGTH:
        pads = tf.fill([batch_size, MAX_SEQUENCE_LENGTH - sequence_length], 0)
        encoder_input_tokens = tf.concat([encoder_input_tokens, pads], 1)

    # Definir una función para la decodificación.
    def next(prompt, cache, index):
        logits = transformer([encoder_input_tokens, prompt])[:, index - 1, :]
        return logits, None, cache

    # Construir un prompt con un token de inicio y tokens de padding.
    length = 40
    start = tf.fill([batch_size, 1], eng_tokenizer.token_to_id("[START]"))
    pad = tf.fill([batch_size, length - 1], eng_tokenizer.token_to_id("[PAD]"))
    prompt = tf.concat([start, pad], axis=-1)

    # Usar el sampler para generar tokens.
    generated_tokens = keras_nlp.samplers.GreedySampler()(
        next,
        prompt,
        end_token_id=eng_tokenizer.token_to_id("[END]"),
        index=1,
    )
    generated_sentences = eng_tokenizer.detokenize(generated_tokens)
    return generated_sentences

#### Iterar sobre las Frases Iniciales y Realizar la Generación y Traducción:

En esta sección del proyecto, nos enfocamos en la generación y traducción de texto, empleando tanto el modelo RNN entrenado como el modelo de traducción Transformer. La metodología utilizada se desglosa en los siguientes pasos:

1. **Selección de Frases Iniciales**: Se define una lista de `start_strings`, que son frases en español utilizadas como punto de partida para la generación de texto. Estas frases iniciales varían en contenido y estructura, permitiendo explorar la diversidad en la generación del modelo RNN.

2. **Generación de Texto en Español**: Para cada frase inicial, se invoca la función `generate_text` con el modelo RNN (`ibai_model`) y la frase de inicio. Esta función produce una secuencia de texto en español que refleja el estilo y tono del modelo RNN entrenado. Se imprime el texto generado para cada frase inicial, permitiendo observar la capacidad del modelo RNN para crear texto coherente y contextualmente relevante.

3. **Traducción al Inglés**: Cada texto generado en español es posteriormente traducido al inglés utilizando la función `decode_sequences`. Esta función se apoya en el modelo Transformer para convertir las secuencias de texto español en su equivalente en inglés. El proceso de traducción considera la coherencia gramatical y semántica, buscando generar traducciones precisas y fluidas.

4. **Presentación de Resultados**: Tras la traducción, se presenta el texto original en español junto con su traducción al inglés. Este proceso se repite para cada una de las frases iniciales, proporcionando una variedad de ejemplos que demuestran la efectividad del modelo RNN en la generación de texto y la capacidad del modelo Transformer para traducir el texto generado.

Este enfoque iterativo de generación y traducción de texto ilustra la sinergia entre dos modelos de aprendizaje profundo avanzados. A través de este proceso, se explora cómo la inteligencia artificial puede no solo crear contenido lingüístico, sino también traducirlo a otro idioma.

In [22]:
start_strings = ["¿ te acuerdas", "hola, buenas", "mi nombre", "¿ tú crees que", "¿ que no hay", "o sea, que"]

for start_string in start_strings:
    print(f"Generando y traduciendo para: '{start_string}'")

    # Generar texto en español
    generated_text = generate_text(ibai_model, start_string)
    print("Texto generado en Español:")
    print(generated_text)
    print()

    # Traducir al inglés
    translated_text = decode_sequences([generated_text])
    translated_text = translated_text.numpy()[0].decode("utf-8").replace("[PAD]", "").replace("[START]", "").replace("[END]", "").strip()
    print("Texto traducido al Inglés:")
    print(translated_text)
    print("-" * 50)

Generando y traduciendo para: '¿ te acuerdas'
Texto generado en Español:
¿ te acuerdas ? algunos sí quedo y que lo q

Texto traducido al Inglés:
do you remember any longer . to stay .
--------------------------------------------------
Generando y traduciendo para: 'hola, buenas'
Texto generado en Español:
hola, buenas tardes seguirás, se ha ido co

Texto traducido al Inglés:
hi , you ' ll be late , never gone .
--------------------------------------------------
Generando y traduciendo para: 'mi nombre'
Texto generado en Español:
mi nombre de cosco, bueno, siré lo que 

Texto traducido al Inglés:
my name is hiding , if you , i ' ll give it .
--------------------------------------------------
Generando y traduciendo para: '¿ tú crees que'
Texto generado en Español:
¿ tú crees que es messi ? ahora hablan del c

Texto traducido al Inglés:
do you think he is now ?
--------------------------------------------------
Generando y traduciendo para: '¿ que no hay'
Texto generado en Español:
¿ que

#### Análisis de Resultados de Generación y Traducción de Texto:

A continuación, se presenta un análisis detallado de los resultados obtenidos en la generación y traducción de texto utilizando el modelo RNN y el modelo Transformer:

1. **Frase Inicial: '¿ te acuerdas'**
   - **Texto en Español**: "¿ te acuerdas ? algunos sí quedo y que lo q"
   - **Traducción al Inglés**: "do you remember any longer . to stay ."
   - **Análisis**: La generación en español capta la esencia interrogativa de la frase inicial, pero la continuidad del texto parece incompleta. La traducción al inglés mantiene la pregunta inicial, pero añade una segunda parte que parece desvinculada del contexto.

2. **Frase Inicial: 'hola, buenas'**
   - **Texto en Español**: "hola, buenas tardes seguirás, se ha ido co"
   - **Traducción al Inglés**: "hi , you ' ll be late , never gone ."
   - **Análisis**: El texto generado en español comienza con un saludo y parece iniciar una conversación, pero termina abruptamente. La traducción al inglés intenta seguir el hilo, aunque presenta una conclusión poco clara.

3. **Frase Inicial: 'mi nombre'**
   - **Texto en Español**: "mi nombre de cosco, bueno, siré lo que"
   - **Traducción al Inglés**: "my name is hiding , if you , i ' ll give it ."
   - **Análisis**: El texto en español comienza con una declaración sobre un nombre, pero no se desarrolla completamente. La traducción al inglés agrega elementos que no están presentes en el original, como "hiding" y "i ' ll give it", lo que indica una interpretación libre del modelo.

4. **Frase Inicial: '¿ tú crees que'**
   - **Texto en Español**: "¿ tú crees que es messi ? ahora hablan del c"
   - **Traducción al Inglés**: "do you think he is now ?"
   - **Análisis**: La generación en español formula una pregunta sobre una figura conocida (Messi) y sugiere un cambio de tema, pero no se completa. La traducción al inglés captura la pregunta pero omite el cambio de tema.

5. **Frase Inicial: '¿ que no hay'**
   - **Texto en Español**: "¿ que no hay huevos ? ahora.  ahora, ahora"
   - **Traducción al Inglés**: "since is there any eggs now ?"
   - **Análisis**: El texto español presenta una pregunta retórica seguida de repetición. La traducción al inglés intenta mantener la pregunta, pero introduce una palabra ("since") que altera el sentido original.

6. **Frase Inicial: 'o sea, que'**
   - **Texto en Español**: "o sea, que  como ven en la tele y un día"
   - **Traducción al Inglés**: "come on to watch , but on tv a day ."
   - **Análisis**: La generación en español comienza una reflexión sobre lo que se ve en la televisión, pero no se desarrolla completamente. La traducción al inglés intenta seguir el contexto, pero agrega elementos que no están en el texto original.

**Conclusión General del resultado**: Los resultados de la generación y traducción de texto demuestran la capacidad del modelo RNN para generar texto coherente y relevante en español, aunque con ciertas limitaciones en la continuidad y coherencia. Por otro lado, el modelo Transformer logra traducir el texto generado al inglés, manteniendo en gran medida el sentido original, aunque en ocasiones introduce elementos que no se alinean completamente con el texto en español. Estos resultados reflejan la complejidad y los desafíos inherentes a la generación automática de texto y su traducción en un contexto de inteligencia artificial.

### Conclusión Detallada del Proyecto de Generación y Traducción de Texto

#### Evaluación General del Proyecto
Este proyecto ha integrado dos componentes clave de la inteligencia artificial en procesamiento de lenguaje natural: la generación de texto utilizando un modelo RNN y la traducción automática con un modelo Transformer. La combinación de estos modelos ha permitido no solo generar texto en español, sino también traducirlo al inglés, demostrando así la versatilidad y capacidad de la IA para abordar tareas lingüísticas complejas.

#### Análisis del Modelo RNN
El modelo RNN, entrenado para imitar el estilo de Ibai Llanos, ha mostrado una habilidad notable para generar texto que refleja patrones de habla y expresiones típicas del personaje. Sin embargo, se observan limitaciones en términos de coherencia y continuidad en los textos generados, una característica común en los modelos de generación de texto basados en caracteres. La generación de texto, aunque creativa y variada, a menudo resulta en fragmentos que carecen de un hilo conductor claro, lo que puede atribuirse a la naturaleza intrínseca del modelo RNN y a la complejidad de emular fielmente el discurso humano.

#### Evaluación del Modelo Transformer para Traducción
El modelo Transformer, implementado para la traducción del texto generado, ha demostrado ser efectivo en la conversión del español al inglés. A pesar de algunas divergencias y adiciones en la traducción que no se encontraban en el texto original, el modelo ha mantenido en su mayoría la esencia y el significado de los textos generados. Esto indica una buena comprensión del contexto y una capacidad notable para manejar la ambigüedad y la variabilidad del lenguaje natural.

#### Desafíos y Limitaciones
Un desafío notable en este proyecto ha sido equilibrar la creatividad y la coherencia en la generación de texto. Mientras que el modelo RNN ha sido capaz de producir texto creativo y diverso, mantener una narrativa coherente y lógica sigue siendo una tarea difícil. Además, la traducción automática, aunque generalmente precisa, a veces introduce elementos que no se alinean completamente con el texto original, lo que puede atribuirse a las limitaciones inherentes a los modelos de traducción automática y a que el texto generado no sea muy coherente o no se acaben las oraciones.

#### Implicaciones y Futuras Direcciones
Los resultados obtenidos abren puertas a exploraciones futuras en la generación de texto y la traducción automática. Existe un amplio margen para mejorar la coherencia y precisión en ambos modelos, posiblemente a través de un entrenamiento más extenso, la utilización de conjuntos de datos más grandes y variados, o la exploración de modelos más avanzados como GPT-3 para la generación de texto. Además, la aplicación de este enfoque en diferentes contextos y estilos de lenguaje podría proporcionar insights valiosos sobre la adaptabilidad y versatilidad de estos modelos de IA.

En conclusión, este proyecto demuestra el potencial y los desafíos de la IA en el campo del procesamiento de lenguaje natural, particularmente en la generación y traducción de texto. Aunque aún hay margen de mejora, los resultados actuales son prometedores e invitan a seguir investigando posibles mejoras.